In [1]:
import requests,re,time,numpy as np,pickle,asyncio,concurrent.futures,datetime
# Make this notebook wiiiiiiiiide
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

This parser works with just a few locations, constructing an ITA-style matrix of price vs start/end dates

In [2]:
# Setup relevant variables
ZIP = '60615'
# AGENCYLIST = [('ET','E11509'), #oak park,
#              #('ET','E11548'), #bolingbrook
#              #('ET','E115CH'), #elmwood
#              #('ET','E11516'), #skokie
#              #('ET','E11553'), #Nn Lincolnwood 53
#             ]
AGENCYLIST = None
RADIUS = '40'

PICKUP = datetime.date(2018, 1, 4)
DROPOFF = PICKUP+datetime.timedelta(days=14)
#DROPOFF = datetime.date(2018, 1, 17)
FIXEDLENGTH = False
VARYSTART = [0,0] # Variance backwards and forwards
VARYEND = [0,0]
TIME = '9:00 AM'

In [3]:
%run backend_withdb_v2.ipynb

In [4]:
if not AGENCYLIST:
    (agencylist,_) = getagencylist((PICKUP,DROPOFF))
    AGENCYLIST = [(x['bcode'],x['code']) for x in agencylist.values()]

In [5]:
AGENCYLIST

[('ET', 'E11586'),
 ('ET', 'E11576'),
 ('AV', 'CHIC05'),
 ('BG', 'CHIC02'),
 ('ET', 'E115DN'),
 ('BG', 'CHIC08'),
 ('ET', 'E11530'),
 ('ET', 'E115DA'),
 ('AV', 'CHIX01'),
 ('BG', 'CHIX01'),
 ('ET', 'E115CE'),
 ('ET', 'E115CD'),
 ('ET', 'E11555'),
 ('ET', 'E115KB'),
 ('AV', 'CHIC01'),
 ('BG', 'CHIC09'),
 ('AL', 'CHIC72'),
 ('ET', 'E115CA'),
 ('ET', 'E115C6'),
 ('ET', 'E115CU'),
 ('ET', 'E11557'),
 ('ET', 'E115C7'),
 ('AV', 'MDWT01'),
 ('BG', 'MDWT02'),
 ('ET', 'E115CQ'),
 ('ET', 'MDWT61'),
 ('AL', 'MDWT71'),
 ('ET', 'E115C4'),
 ('AV', 'CHIC12'),
 ('BG', 'CHIC12'),
 ('AV', 'CHIS08'),
 ('BG', 'CHIS04'),
 ('ET', 'E115MT'),
 ('ET', 'E11503'),
 ('ET', 'E115CF'),
 ('ET', 'E11578'),
 ('ET', 'E11575'),
 ('ET', 'E1159D'),
 ('ET', 'E115FX'),
 ('ET', 'E11510'),
 ('AV', 'CHIC04'),
 ('BG', 'CHIC01'),
 ('BG', 'CHIC07'),
 ('AV', 'CHIC07'),
 ('AV', 'CHIS04'),
 ('BG', 'CHIS02'),
 ('ET', 'E115C9'),
 ('AV', 'CHIW12'),
 ('BG', 'CHIW09'),
 ('ET', 'E11509'),
 ('ET', 'E11588'),
 ('ET', 'E11581'),
 ('ET', 'E11

In [6]:
populateagenciesdb((PICKUP,DROPOFF))

In [7]:
runparser(agencylist=AGENCYLIST, threads=1);

034 ND |  01/04/2018 01/18/2018 35 enterprise E115CF Sat Howard Orloff ET 8.76
038 ND |  01/04/2018 01/18/2018 39 enterprise E115FX Nn Fox Ford ET 9.58
093 ND |  01/04/2018 01/18/2018 94 enterprise E115P1 Packer Branch Elmhurst ET 19.34
107 ND |  01/04/2018 01/18/2018 108 enterprise E115PD Packer Br Downers Grove ET 21.30
181 ND |  01/04/2018 01/18/2018 182 enterprise E115PA Packer Branch Joliet ET 33.51


In [8]:
res = session.query(Price.car_econ, Price.pickup, Price.dropoff, Agency.ag_dist, Agency.ag_brand, Agency.ag_name).join(Agency).filter(Price.car_econ.isnot(None)).order_by(Price.car_econ).all()

In [9]:
for i,v in enumerate(res):
    print('{:4d} | {:5.2f} | {:5s} | {:5s} | {:6.2f} | {:10s} | {:40s}'
              .format(i,v[0],v[1],v[2],v[3],v[4],v[5]))

   0 | 211.93 | 01/04/2018 | 01/18/2018 |  24.53 | enterprise | Aa Glen Ellyn                           
   1 | 213.63 | 01/04/2018 | 01/18/2018 |  21.31 | enterprise | Aa South Downers Grove                  
   2 | 215.12 | 01/04/2018 | 01/18/2018 |  26.20 | enterprise | Aa Lockport                             
   3 | 215.12 | 01/04/2018 | 01/18/2018 |  35.82 | enterprise | Aa Shorewood                            
   4 | 217.17 | 01/04/2018 | 01/18/2018 |  32.26 | enterprise | Aa Crest Hill                           
   5 | 217.17 | 01/04/2018 | 01/18/2018 |  33.53 | enterprise | Aa Joliet                               
   6 | 217.65 | 01/04/2018 | 01/18/2018 |  19.34 | enterprise | Aa Elmhurst                             
   7 | 217.65 | 01/04/2018 | 01/18/2018 |  19.86 | enterprise | Aa Lombard                              
   8 | 217.65 | 01/04/2018 | 01/18/2018 |  20.15 | enterprise | Aa W Grand Ave                          
   9 | 217.65 | 01/04/2018 | 01/18/2018 |  20.90 | ente

In [ ]:
"""
  0 | 213.63 | 01/05/2018 | 01/19/2018 |  21.31 | enterprise | Aa South Downers Grove                  
   1 | 219.23 | 01/05/2018 | 01/19/2018 |  20.30 | budget     | Lombard                                 
   2 | 225.72 | 01/05/2018 | 01/19/2018 |  13.64 | enterprise | Aa Lagrange                             
   3 | 228.44 | 01/05/2018 | 01/19/2018 |  18.03 | budget     | Oak Brook Il                            
   4 | 228.83 | 01/05/2018 | 01/19/2018 |  24.83 | enterprise | Aa Lisle                                
   5 | 228.83 | 01/05/2018 | 01/19/2018 |  27.09 | enterprise | Aa East Naperville                      
   6 | 229.31 | 01/05/2018 | 01/19/2018 |  24.53 | enterprise | Aa Glen Ellyn                           
   7 | 230.61 | 01/05/2018 | 01/19/2018 |  35.80 | budget     | St.charles                              
   8 | 230.61 | 01/05/2018 | 01/19/2018 |  23.57 | budget     | Addison                                 
   9 | 230.61 | 01/05/2018 | 01/19/2018 |  36.89 | budget     | Aurora                                  
  10 | 230.61 | 01/05/2018 | 01/19/2018 |  24.47 | budget     | The Hyatt Of Lisle Il                   
  11 | 230.61 | 01/05/2018 | 01/19/2018 |  27.01 | budget     | Naperville                              
  12 | 230.61 | 01/05/2018 | 01/19/2018 |  28.59 | budget     | Romeoville, Il Lewis Airport            
  13 | 230.61 | 01/05/2018 | 01/19/2018 |  30.94 | budget     | Naperville Il                           
  14 | 230.61 | 01/05/2018 | 01/19/2018 |  31.24 | budget     | Aurora Il Sears                         
  15 | 230.61 | 01/05/2018 | 01/19/2018 |  32.83 | budget     | Joliet Il Sears                         
  16 | 230.61 | 01/05/2018 | 01/19/2018 |  21.35 | budget     | Downers Grove Il                        
  17 | 230.72 | 01/05/2018 | 01/19/2018 |  24.81 | enterprise | Aa Bolingbrook                          
  18 | 232.89 | 01/05/2018 | 01/19/2018 |  32.22 | enterprise | Aa South Naperville                     
  19 | 232.89 | 01/05/2018 | 01/19/2018 |  32.46 | enterprise | Aa North Plainfield                     
  20 | 232.89 | 01/05/2018 | 01/19/2018 |  35.69 | enterprise | Aa Plainfield                           
  21 | 232.92 | 01/05/2018 | 01/19/2018 |  30.14 | enterprise | Aa West Ogden                           
  22 | 232.92 | 01/05/2018 | 01/19/2018 |  30.85 | enterprise | Aa Warrenville                          
  23 | 232.92 | 01/05/2018 | 01/19/2018 |  31.04 | enterprise | Aa Naperville North                     
  24 | 232.92 | 01/05/2018 | 01/19/2018 |  31.67 | enterprise | Aa East Aurora                          
  25 | 236.39 | 01/05/2018 | 01/19/2018 |  18.57 | enterprise | Aa Willowbrook                          
  26 | 237.57 | 01/05/2018 | 01/19/2018 |  18.03 | avis       | Oak Brook Doubletree Hotel              
  27 | 237.57 | 01/05/2018 | 01/19/2018 |  20.30 | avis       | Lombard                                 
  28 | 238.64 | 01/05/2018 | 01/19/2018 |  21.41 | enterprise | Aa North Downers Grove                  
  29 | 238.64 | 01/05/2018 | 01/19/2018 |  19.16 | enterprise | Aa Westmont                             
  30 | 238.66 | 01/05/2018 | 01/19/2018 |  27.68 | enterprise | Aa Carol Stream                         
  31 | 239.84 | 01/05/2018 | 01/19/2018 |  36.89 | avis       | Aurora                                  
  32 | 239.84 | 01/05/2018 | 01/19/2018 |  23.70 | avis       | Addison                                 
  33 | 239.84 | 01/05/2018 | 01/19/2018 |  24.48 | avis       | The Hyatt Of Lisle                      
  34 | 239.84 | 01/05/2018 | 01/19/2018 |  27.04 | avis       | Naperville, Il                          
  35 | 239.84 | 01/05/2018 | 01/19/2018 |  28.59 | avis       | Lewis University Airport                
  36 | 239.84 | 01/05/2018 | 01/19/2018 |  30.94 | avis       | Naperville - Pebblewood Plaza           
  37 | 239.84 | 01/05/2018 | 01/19/2018 |  31.24 | avis       | Sears Auto Ctr-westfield Fox Valley Mall
  38 | 239.84 | 01/05/2018 | 01/19/2018 |  32.83 | avis       | Sears Auto Center - Louis Joliet Mall   
  39 | 239.84 | 01/05/2018 | 01/19/2018 |  21.35 | avis       | Downers Grove, Il                       
  40 | 239.84 | 01/05/2018 | 01/19/2018 |  35.80 | avis       | Foxfield Commons Shopping Plaza         
  41 | 239.93 | 01/05/2018 | 01/19/2018 |  10.97 | enterprise | Dd Oak Park                             
  42 | 239.93 | 01/05/2018 | 01/19/2018 |  14.15 | enterprise | Dd Elmwood Park                         
  43 | 239.93 | 01/05/2018 | 01/19/2018 |  16.04 | enterprise | Dd Stone Park                           
  44 | 243.67 | 01/05/2018 | 01/19/2018 |  10.78 | budget     | Chicago Ridge Il Sears                  
  45 | 243.67 | 01/05/2018 | 01/19/2018 |  16.99 | budget     | Orland Park                             
  46 | 243.67 | 01/05/2018 | 01/19/2018 |   8.04 | budget     | Oak Lawn                                
  47 | 244.37 | 01/05/2018 | 01/19/2018 |  19.34 | enterprise | Aa Elmhurst                             
  48 | 244.37 | 01/05/2018 | 01/19/2018 |  19.86 | enterprise | Aa Lombard                              
  49 | 244.37 | 01/05/2018 | 01/19/2018 |  20.15 | enterprise | Aa W Grand Ave                          
  50 | 244.37 | 01/05/2018 | 01/19/2018 |  20.90 | enterprise | Aa Villa Park                           
  51 | 245.63 | 01/05/2018 | 01/19/2018 |  37.15 | enterprise | Aa Barrington                           
  52 | 249.06 | 01/05/2018 | 01/19/2018 |  37.18 | budget     | Lake Zurich Il                          
  53 | 249.06 | 01/05/2018 | 01/19/2018 |  39.56 | budget     | Libertyville Il                         
  54 | 249.06 | 01/05/2018 | 01/19/2018 |  29.21 | budget     | Highland Park                           
  55 | 251.58 | 01/05/2018 | 01/19/2018 |  27.93 | enterprise | Nn-hyatt-deerfield                      
  56 | 253.41 | 01/05/2018 | 01/19/2018 |  10.78 | avis       | Sears Auto-westfield Chicago Ridge Mall 
  57 | 253.41 | 01/05/2018 | 01/19/2018 |  16.99 | avis       | Orland Park, Il                         
  58 | 253.41 | 01/05/2018 | 01/19/2018 |   8.04 | avis       | Oak Lawn/chicago                        
  59 | 253.98 | 01/05/2018 | 01/19/2018 |  28.39 | enterprise | Nn-highland Park                        
  60 | 255.29 | 01/05/2018 | 01/19/2018 |  18.96 | enterprise | Aa Holiday Inn                          
  61 | 255.59 | 01/05/2018 | 01/19/2018 |  17.76 | enterprise | Nn South Evanston N2                    
  62 | 256.18 | 01/05/2018 | 01/19/2018 |  16.04 | enterprise | Nn Lincolnwood 53                       
  63 | 256.18 | 01/05/2018 | 01/19/2018 |  17.54 | enterprise | Nn Skokie                               
  64 | 256.18 | 01/05/2018 | 01/19/2018 |  18.86 | enterprise | Morton Grove                            
  65 | 256.24 | 01/05/2018 | 01/19/2018 |  18.96 | enterprise | Nn North Evanston                       
  66 | 258.34 | 01/05/2018 | 01/19/2018 |  35.82 | enterprise | Aa Shorewood                            
  67 | 258.34 | 01/05/2018 | 01/19/2018 |  26.20 | enterprise | Aa Lockport                             
  68 | 258.91 | 01/05/2018 | 01/19/2018 |  36.51 | enterprise | Nn-mundelein                            
  69 | 259.53 | 01/05/2018 | 01/19/2018 |  32.84 | enterprise | Aa Palatine                             
  70 | 260.80 | 01/05/2018 | 01/19/2018 |  32.26 | enterprise | Aa Crest Hill                           
  71 | 260.80 | 01/05/2018 | 01/19/2018 |  33.53 | enterprise | Aa Joliet                               
  72 | 261.36 | 01/05/2018 | 01/19/2018 |  36.83 | enterprise | Aa East St. Charles                     
  73 | 261.36 | 01/05/2018 | 01/19/2018 |  38.45 | enterprise | Aa St Charles                           
  74 | 261.38 | 01/05/2018 | 01/19/2018 |  37.14 | enterprise | Nn-libertyville                         
  75 | 263.16 | 01/05/2018 | 01/19/2018 |  25.02 | budget     | Northbrook                              
  76 | 266.93 | 01/05/2018 | 01/19/2018 |  39.05 | enterprise | Aa East Dundee                          
  77 | 267.50 | 01/05/2018 | 01/19/2018 |  37.18 | avis       | Lake Zurich-village Square Auto Center  
  78 | 267.50 | 01/05/2018 | 01/19/2018 |  39.56 | avis       | Libertyville - Peterson Road            
  79 | 267.50 | 01/05/2018 | 01/19/2018 |  29.21 | avis       | Highland Park Ford                      
  80 | 268.36 | 01/05/2018 | 01/19/2018 |  26.13 | enterprise | Nn-wheeling                             
  81 | 268.36 | 01/05/2018 | 01/19/2018 |  30.91 | enterprise | Nn-napleton Dge Jeep Chr                
  82 | 270.37 | 01/05/2018 | 01/19/2018 |   8.08 | enterprise | Euro Collission                         
  83 | 274.21 | 01/05/2018 | 01/19/2018 |  10.78 | enterprise | Nn Gerber Logan Square                  
  84 | 274.21 | 01/05/2018 | 01/19/2018 |  11.92 | enterprise | Nn Cragin                               
  85 | 274.21 | 01/05/2018 | 01/19/2018 |  13.31 | enterprise | Nn Irving Park                          
  86 | 279.52 | 01/05/2018 | 01/19/2018 |  37.33 | enterprise | Aa Elgin                                
  87 | 279.52 | 01/05/2018 | 01/19/2018 |  15.24 | enterprise | Nn Norwood Park C1                      
  88 | 279.58 | 01/05/2018 | 01/19/2018 |  11.85 | enterprise | Nn Riverview                            
  89 | 279.58 | 01/05/2018 | 01/19/2018 |  13.81 | enterprise | Napleton Chrysler                       
  90 | 280.12 | 01/05/2018 | 01/19/2018 |  10.82 | budget     | Oak Park                                
  91 | 280.12 | 01/05/2018 | 01/19/2018 |  31.61 | budget     | Streamwood Il                           
  92 | 282.04 | 01/05/2018 | 01/19/2018 |  30.52 | enterprise | Aa Hanover Park                         
  93 | 282.05 | 01/05/2018 | 01/19/2018 |  10.82 | avis       | Oak Park                                
  94 | 282.05 | 01/05/2018 | 01/19/2018 |  31.61 | avis       | Streamwood, Il                          
  95 | 282.64 | 01/05/2018 | 01/19/2018 |  22.76 | budget     | Elk Grove Village Il                    
  96 | 282.64 | 01/05/2018 | 01/19/2018 |  25.02 | avis       | Renaissance Chicago North Shore         
  97 | 282.64 | 01/05/2018 | 01/19/2018 |  15.03 | budget     | Lincolnwood                             
  98 | 282.64 | 01/05/2018 | 01/19/2018 |  16.46 | budget     | Evanston                                
  99 | 282.64 | 01/05/2018 | 01/19/2018 |  18.35 | budget     | Skokie                                  
 100 | 282.64 | 01/05/2018 | 01/19/2018 |  21.15 | budget     | Niles Il Sears                          
 101 | 283.64 | 01/05/2018 | 01/19/2018 |  28.09 | budget     | Hyatt Woodfield                         
 102 | 283.64 | 01/05/2018 | 01/19/2018 |  30.75 | budget     | Schaumburg                              
 103 | 284.59 | 01/05/2018 | 01/19/2018 |  22.76 | avis       | Elk Grove Village, Il                   
 104 | 284.59 | 01/05/2018 | 01/19/2018 |  16.46 | avis       | Evanston (525 Kedzie Building)          
 105 | 284.59 | 01/05/2018 | 01/19/2018 |  18.35 | avis       | Skokie - Dempster Street                
 106 | 284.59 | 01/05/2018 | 01/19/2018 |  21.15 | avis       | Sears Auto Ctr - Golf Mill Shopping Ctr 
 107 | 285.59 | 01/05/2018 | 01/19/2018 |  28.09 | avis       | Schaumburg Hyatt Regency Hotel          
 108 | 285.59 | 01/05/2018 | 01/19/2018 |  30.75 | avis       | Schaumburg/hoffman Estates              
 109 | 301.33 | 01/05/2018 | 01/19/2018 |  21.60 | enterprise | Gg Gary                                 
 110 | 301.33 | 01/05/2018 | 01/19/2018 |  14.69 | enterprise | Gg Hammond                              
 111 | 302.54 | 01/05/2018 | 01/19/2018 |  23.02 | enterprise | Gg Steger                               
 112 | 302.54 | 01/05/2018 | 01/19/2018 |  26.83 | enterprise | Gg New Lenox                            
 113 | 302.59 | 01/05/2018 | 01/19/2018 |  36.35 | enterprise | Aa Oswego                               
 114 | 305.35 | 01/05/2018 | 01/19/2018 |  21.81 | enterprise | Gg Schererville                         
 115 | 305.35 | 01/05/2018 | 01/19/2018 |  36.52 | enterprise | Gg Valparaiso                           
 116 | 305.35 | 01/05/2018 | 01/19/2018 |  24.13 | enterprise | Gg Hobart                               
 117 | 305.35 | 01/05/2018 | 01/19/2018 |  26.33 | enterprise | Gg Merrillville                         
 118 | 305.35 | 01/05/2018 | 01/19/2018 |  28.45 | enterprise | Gg East Merrillville                    
 119 | 305.35 | 01/05/2018 | 01/19/2018 |  29.60 | enterprise | Gg Crown Point Indiana                  
 120 | 305.35 | 01/05/2018 | 01/19/2018 |  30.13 | enterprise | Gg Cedar Lake                           
 121 | 305.35 | 01/05/2018 | 01/19/2018 |  19.43 | enterprise | Gg Highland                             
 122 | 305.35 | 01/05/2018 | 01/19/2018 |  19.55 | enterprise | Gg Griffith                             
 123 | 305.42 | 01/05/2018 | 01/19/2018 |  22.13 | enterprise | Gg Frankfort And Mokena                 
 124 | 305.47 | 01/05/2018 | 01/19/2018 |  36.65 | enterprise | Aa Aurora                               
 125 | 308.11 | 01/05/2018 | 01/19/2018 |  10.15 | budget     | Logan Square Chicago Il                 
 126 | 308.11 | 01/05/2018 | 01/19/2018 |  10.55 | budget     | Chicago/lakeview                        
 127 | 308.11 | 01/05/2018 | 01/19/2018 |   5.11 | budget     | Chicago                                 
 128 | 308.11 | 01/05/2018 | 01/19/2018 |   7.94 | budget     | Chicago Grossinger Autoplex             
 129 | 308.11 | 01/05/2018 | 01/19/2018 |   4.05 | budget     | Chicago                                 
 130 | 310.21 | 01/05/2018 | 01/19/2018 |  10.56 | avis       | Lakeview/chicago                        
 131 | 310.21 | 01/05/2018 | 01/19/2018 |   5.71 | avis       | Chicago Union Train Station             
 132 | 310.21 | 01/05/2018 | 01/19/2018 |   7.94 | avis       | Grossinger City Autoplex                
 133 | 310.21 | 01/05/2018 | 01/19/2018 |  10.15 | avis       | Logan Square                            
 134 | 310.21 | 01/05/2018 | 01/19/2018 |   4.05 | avis       | Chicago - 18th Street & Wabash Avenue   
 135 | 313.61 | 01/05/2018 | 01/19/2018 |  37.57 | budget     | Libertyville                            
 136 | 313.71 | 01/05/2018 | 01/19/2018 |  27.52 | enterprise | Aa Rolling Meadows Nt                   
 137 | 316.54 | 01/05/2018 | 01/19/2018 |  27.34 | enterprise | Nn-arlington Heights                    
 138 | 317.54 | 01/05/2018 | 01/19/2018 |  27.81 | enterprise | Aa East Schaumburg                      
 139 | 317.54 | 01/05/2018 | 01/19/2018 |  28.43 | enterprise | Aa Roselle                              
 140 | 317.54 | 01/05/2018 | 01/19/2018 |  28.87 | enterprise | Aa Schaumburg                           
 141 | 317.54 | 01/05/2018 | 01/19/2018 |  30.76 | enterprise | Aa Hoffman Estates                      
 142 | 319.83 | 01/05/2018 | 01/19/2018 |  22.80 | enterprise | Nn-glenview                             
 143 | 319.83 | 01/05/2018 | 01/19/2018 |  12.70 | enterprise | Gg Brookfield                           
 144 | 319.83 | 01/05/2018 | 01/19/2018 |  14.33 | enterprise | Gg Calumet City                         
 145 | 319.83 | 01/05/2018 | 01/19/2018 |   9.33 | enterprise | Gg Berwyn                               
 146 | 319.83 | 01/05/2018 | 01/19/2018 |   9.47 | enterprise | Gg Blue Island                          
 147 | 321.33 | 01/05/2018 | 01/19/2018 |  28.25 | enterprise | Gg Portage                              
 148 | 321.33 | 01/05/2018 | 01/19/2018 |  28.80 | enterprise | Gg Arnell                               
 149 | 322.71 | 01/05/2018 | 01/19/2018 |  21.92 | enterprise | Gg Miller Automall                      
 150 | 322.71 | 01/05/2018 | 01/19/2018 |  11.09 | enterprise | Gg Bridgeview                           
 151 | 322.71 | 01/05/2018 | 01/19/2018 |  11.20 | enterprise | Gg Crestwood                            
 152 | 322.71 | 01/05/2018 | 01/19/2018 |  11.71 | enterprise | Gg Forest Park                          
 153 | 322.71 | 01/05/2018 | 01/19/2018 |  13.76 | enterprise | Gg South Holland                        
 154 | 322.71 | 01/05/2018 | 01/19/2018 |  16.75 | enterprise | Gg Orland Park East                     
 155 | 322.71 | 01/05/2018 | 01/19/2018 |  17.26 | enterprise | Gg Homewood                             
 156 | 322.71 | 01/05/2018 | 01/19/2018 |  18.03 | enterprise | Gg Lansing                              
 157 | 322.71 | 01/05/2018 | 01/19/2018 |   8.70 | enterprise | Gg Oak Lawn                             
 158 | 322.71 | 01/05/2018 | 01/19/2018 |  21.35 | enterprise | Gg Matteson                             
 159 | 323.71 | 01/05/2018 | 01/19/2018 |  17.63 | enterprise | Schiller Park                           
 160 | 325.46 | 01/05/2018 | 01/19/2018 |  18.12 | enterprise | Gg Orland West                          
 161 | 332.06 | 01/05/2018 | 01/19/2018 |  37.57 | avis       | Closing February 1, 2018                
 162 | 338.65 | 01/05/2018 | 01/19/2018 |   5.71 | budget     | Chicago Union Train Station             
 163 | 340.52 | 01/05/2018 | 01/19/2018 |   5.82 | enterprise | Chicago Loop                            
 164 | 340.52 | 01/05/2018 | 01/19/2018 |   5.84 | enterprise | Dd West Loop A/b(ex15ca)                
 165 | 340.52 | 01/05/2018 | 01/19/2018 |   5.93 | enterprise | Union Station/pres Tower                
 166 | 343.63 | 01/05/2018 | 01/19/2018 |  37.77 | enterprise | Nn-north Chicago                        
 167 | 348.76 | 01/05/2018 | 01/19/2018 |  19.01 | budget     | Ohare Intl Airport                      
 168 | 351.39 | 01/05/2018 | 01/19/2018 |   4.19 | enterprise | Gg South Western                        
 169 | 351.39 | 01/05/2018 | 01/19/2018 |  12.59 | enterprise | Nn Uptown                               
 170 | 351.39 | 01/05/2018 | 01/19/2018 |   5.17 | enterprise | Gg Stony Island                         
 171 | 351.39 | 01/05/2018 | 01/19/2018 |   5.57 | enterprise | Gg Gresham                              
 172 | 351.39 | 01/05/2018 | 01/19/2018 |   6.13 | enterprise | Gg Archer Heights                       
 173 | 351.39 | 01/05/2018 | 01/19/2018 |   6.17 | enterprise | Gg Western And Roosevelt                
 174 | 351.39 | 01/05/2018 | 01/19/2018 |   7.54 | enterprise | Dd Gold Coast Auto Body                 
 175 | 351.39 | 01/05/2018 | 01/19/2018 |   7.87 | enterprise | Dd Old Town                             
 176 | 351.39 | 01/05/2018 | 01/19/2018 |   8.91 | enterprise | Dd Bucktown                             
 177 | 351.39 | 01/05/2018 | 01/19/2018 |   3.17 | enterprise | Gg South Loop                           
 178 | 351.39 | 01/05/2018 | 01/19/2018 |   9.61 | enterprise | Nn Lincoln Park                         
 179 | 363.03 | 01/05/2018 | 01/19/2018 |  23.87 | enterprise | Nn-mount Prospect                       
 180 | 363.03 | 01/05/2018 | 01/19/2018 |  20.19 | enterprise | Nn-park Ridge                           
 181 | 363.03 | 01/05/2018 | 01/19/2018 |  20.79 | enterprise | Nn Des Plaines 1505                     
 182 | 363.03 | 01/05/2018 | 01/19/2018 |  21.00 | enterprise | Golf Mill Ford                          
 183 | 368.82 | 01/05/2018 | 01/19/2018 |   0.95 | enterprise | Gg Hyde Park                            
 184 | 382.44 | 01/05/2018 | 01/19/2018 |  26.05 | budget     | Merrillville                            
 185 | 392.09 | 01/05/2018 | 01/19/2018 |  26.05 | avis       | Merrillville Car Care Center            
 186 | 395.23 | 01/05/2018 | 01/19/2018 |  39.53 | enterprise | Gg Michigan City                        
 187 | 398.66 | 01/05/2018 | 01/19/2018 |   6.05 | budget     | North Loop Budget                       
 188 | 419.72 | 01/05/2018 | 01/19/2018 |   6.05 | avis       | North Loop-downtown                     
 189 | 441.80 | 01/05/2018 | 01/19/2018 |   5.93 | enterprise | Millennium Park                         
 190 | 441.80 | 01/05/2018 | 01/19/2018 |   6.09 | enterprise | Lake And Franklin                       
 191 | 441.80 | 01/05/2018 | 01/19/2018 |   6.43 | enterprise | Chicago Magnificent Mile                
 192 | 441.80 | 01/05/2018 | 01/19/2018 |   6.86 | enterprise | Gold Coast                              
 193 | 466.95 | 01/05/2018 | 01/19/2018 |   6.07 | alamo      | Chicago N Lasalle                       
 194 | 584.01 | 01/05/2018 | 01/19/2018 |   7.54 | budget     | Midway Airport                          
 195 | 709.02 | 01/05/2018 | 01/19/2018 |  19.06 | alamo      | Chicago Ohare Airport                   
 196 | 709.02 | 01/05/2018 | 01/19/2018 |  19.79 | enterprise | Ohare Intl Arpt                         
 197 | 716.01 | 01/05/2018 | 01/19/2018 |  19.01 | avis       | Chicago Ohare Intl Airport              
 198 | 765.67 | 01/05/2018 | 01/19/2018 |   7.60 | alamo      | Midway Intl Arpt                        
 199 | 790.57 | 01/05/2018 | 01/19/2018 |   7.54 | avis       | Chicago Midway Intl Airport             
 200 | 814.88 | 01/05/2018 | 01/19/2018 |   7.60 | enterprise | Midway Intl Arpt                          
"""

In [ ]:
res = session.query(Price.car_int, Price.pickup, Agency.ag_dist, Agency.ag_brand, Agency.ag_name).join(Agency).filter(Price.car_int.isnot(None)).order_by(Price.car_int).all()

In [ ]:
for i,v in enumerate(res):
    print('{:4d} | {:5.2f} | {:5s} | {:6.2f} | {:10s} | {:40s}'
              .format(i,v[0],v[1],v[2],v[3],v[4]))

In [ ]:
"""
 0 | 237.57 | 01/05/2018 |  18.03 | budget     | Oak Brook Il                            
   1 | 237.57 | 01/05/2018 |  20.30 | budget     | Lombard                                 
   2 | 239.84 | 01/05/2018 |  35.80 | budget     | St.charles                              
   3 | 239.84 | 01/05/2018 |  23.57 | budget     | Addison                                 
   4 | 239.84 | 01/05/2018 |  36.89 | budget     | Aurora                                  
   5 | 239.84 | 01/05/2018 |  24.47 | budget     | The Hyatt Of Lisle Il                   
   6 | 239.84 | 01/05/2018 |  27.01 | budget     | Naperville                              
   7 | 239.84 | 01/05/2018 |  28.59 | budget     | Romeoville, Il Lewis Airport            
   8 | 239.84 | 01/05/2018 |  30.94 | budget     | Naperville Il                           
   9 | 239.84 | 01/05/2018 |  31.24 | budget     | Aurora Il Sears                         
  10 | 239.84 | 01/05/2018 |  32.83 | budget     | Joliet Il Sears                         
  11 | 239.84 | 01/05/2018 |  21.35 | budget     | Downers Grove Il                        
  12 | 242.77 | 01/05/2018 |  21.31 | enterprise | Aa South Downers Grove                  
  13 | 246.71 | 01/05/2018 |  18.03 | avis       | Oak Brook Doubletree Hotel              
  14 | 246.71 | 01/05/2018 |  20.30 | avis       | Lombard                                 
  15 | 249.06 | 01/05/2018 |  36.89 | avis       | Aurora                                  
  16 | 249.06 | 01/05/2018 |  23.70 | avis       | Addison                                 
  17 | 249.06 | 01/05/2018 |  24.48 | avis       | The Hyatt Of Lisle                      
  18 | 249.06 | 01/05/2018 |  27.04 | avis       | Naperville, Il                          
  19 | 249.06 | 01/05/2018 |  28.59 | avis       | Lewis University Airport                
  20 | 249.06 | 01/05/2018 |  30.94 | avis       | Naperville - Pebblewood Plaza           
  21 | 249.06 | 01/05/2018 |  31.24 | avis       | Sears Auto Ctr-westfield Fox Valley Mall
  22 | 249.06 | 01/05/2018 |  32.83 | avis       | Sears Auto Center - Louis Joliet Mall   
  23 | 249.06 | 01/05/2018 |  21.35 | avis       | Downers Grove, Il                       
  24 | 249.06 | 01/05/2018 |  35.80 | avis       | Foxfield Commons Shopping Plaza         
  25 | 252.47 | 01/05/2018 |  37.15 | enterprise | Aa Barrington                           
  26 | 253.41 | 01/05/2018 |  10.78 | budget     | Chicago Ridge Il Sears                  
  27 | 253.41 | 01/05/2018 |  16.99 | budget     | Orland Park                             
  28 | 253.41 | 01/05/2018 |   8.04 | budget     | Oak Lawn                                
  29 | 256.51 | 01/05/2018 |  13.64 | enterprise | Aa Lagrange                             
  30 | 257.25 | 01/05/2018 |  27.93 | enterprise | Nn-hyatt-deerfield                      
  31 | 259.70 | 01/05/2018 |  28.39 | enterprise | Nn-highland Park                        
  32 | 259.89 | 01/05/2018 |  24.53 | enterprise | Aa Glen Ellyn                           
  33 | 259.89 | 01/05/2018 |  24.83 | enterprise | Aa Lisle                                
  34 | 259.89 | 01/05/2018 |  27.09 | enterprise | Aa East Naperville                      
  35 | 261.31 | 01/05/2018 |  24.81 | enterprise | Aa Bolingbrook                          
  36 | 263.16 | 01/05/2018 |  10.78 | avis       | Sears Auto-westfield Chicago Ridge Mall 
  37 | 263.16 | 01/05/2018 |  16.99 | avis       | Orland Park, Il                         
  38 | 263.16 | 01/05/2018 |   8.04 | avis       | Oak Lawn/chicago                        
  39 | 263.47 | 01/05/2018 |  32.22 | enterprise | Aa South Naperville                     
  40 | 263.47 | 01/05/2018 |  32.46 | enterprise | Aa North Plainfield                     
  41 | 263.47 | 01/05/2018 |  35.69 | enterprise | Aa Plainfield                           
  42 | 263.50 | 01/05/2018 |  30.14 | enterprise | Aa West Ogden                           
  43 | 263.50 | 01/05/2018 |  30.85 | enterprise | Aa Warrenville                          
  44 | 263.50 | 01/05/2018 |  31.04 | enterprise | Aa Naperville North                     
  45 | 263.50 | 01/05/2018 |  31.67 | enterprise | Aa East Aurora                          
  46 | 264.58 | 01/05/2018 |  36.51 | enterprise | Nn-mundelein                            
  47 | 265.65 | 01/05/2018 |  35.82 | enterprise | Aa Shorewood                            
  48 | 265.65 | 01/05/2018 |  26.20 | enterprise | Aa Lockport                             
  49 | 266.68 | 01/05/2018 |  18.57 | enterprise | Aa Willowbrook                          
  50 | 266.76 | 01/05/2018 |  32.84 | enterprise | Aa Palatine                             
  51 | 267.10 | 01/05/2018 |  37.14 | enterprise | Nn-libertyville                         
  52 | 267.50 | 01/05/2018 |  37.18 | budget     | Lake Zurich Il                          
  53 | 267.50 | 01/05/2018 |  39.56 | budget     | Libertyville Il                         
  54 | 267.50 | 01/05/2018 |  29.21 | budget     | Highland Park                           
  55 | 268.18 | 01/05/2018 |  36.83 | enterprise | Aa East St. Charles                     
  56 | 268.18 | 01/05/2018 |  38.45 | enterprise | Aa St Charles                           
  57 | 268.18 | 01/05/2018 |  32.26 | enterprise | Aa Crest Hill                           
  58 | 268.18 | 01/05/2018 |  33.53 | enterprise | Aa Joliet                               
  59 | 269.22 | 01/05/2018 |  21.41 | enterprise | Aa North Downers Grove                  
  60 | 269.22 | 01/05/2018 |  19.16 | enterprise | Aa Westmont                             
  61 | 269.24 | 01/05/2018 |  27.68 | enterprise | Aa Carol Stream                         
  62 | 271.17 | 01/05/2018 |  18.96 | enterprise | Aa Holiday Inn                          
  63 | 271.95 | 01/05/2018 |  10.97 | enterprise | Dd Oak Park                             
  64 | 271.95 | 01/05/2018 |  14.15 | enterprise | Dd Elmwood Park                         
  65 | 271.95 | 01/05/2018 |  16.04 | enterprise | Dd Stone Park                           
  66 | 273.75 | 01/05/2018 |  39.05 | enterprise | Aa East Dundee                          
  67 | 274.40 | 01/05/2018 |  26.13 | enterprise | Nn-wheeling                             
  68 | 274.40 | 01/05/2018 |  30.91 | enterprise | Nn-napleton Dge Jeep Chr                
  69 | 282.64 | 01/05/2018 |  25.02 | budget     | Northbrook                              
  70 | 284.48 | 01/05/2018 |  19.34 | enterprise | Aa Elmhurst                             
  71 | 284.48 | 01/05/2018 |  19.86 | enterprise | Aa Lombard                              
  72 | 284.48 | 01/05/2018 |  20.15 | enterprise | Aa W Grand Ave                          
  73 | 284.48 | 01/05/2018 |  20.90 | enterprise | Aa Villa Park                           
  74 | 285.95 | 01/05/2018 |  37.18 | avis       | Lake Zurich-village Square Auto Center  
  75 | 285.95 | 01/05/2018 |  39.56 | avis       | Libertyville - Peterson Road            
  76 | 285.95 | 01/05/2018 |  29.21 | avis       | Highland Park Ford                      
  77 | 286.67 | 01/05/2018 |  37.33 | enterprise | Aa Elgin                                
  78 | 286.97 | 01/05/2018 |   7.94 | avis       | Grossinger City Autoplex                
  79 | 287.90 | 01/05/2018 |  17.76 | enterprise | Nn South Evanston N2                    
  80 | 288.49 | 01/05/2018 |  16.04 | enterprise | Nn Lincolnwood 53                       
  81 | 288.49 | 01/05/2018 |  17.54 | enterprise | Nn Skokie                               
  82 | 288.49 | 01/05/2018 |  18.86 | enterprise | Morton Grove                            
  83 | 288.49 | 01/05/2018 |  18.96 | enterprise | Nn North Evanston                       
  84 | 289.25 | 01/05/2018 |  30.52 | enterprise | Aa Hanover Park                         
  85 | 298.22 | 01/05/2018 |  10.78 | enterprise | Nn Gerber Logan Square                  
  86 | 298.22 | 01/05/2018 |  11.92 | enterprise | Nn Cragin                               
  87 | 298.22 | 01/05/2018 |  13.31 | enterprise | Nn Irving Park                          
  88 | 299.20 | 01/05/2018 |   8.08 | enterprise | Euro Collission                         
  89 | 299.43 | 01/05/2018 |  10.82 | budget     | Oak Park                                
  90 | 299.43 | 01/05/2018 |  31.61 | budget     | Streamwood Il                           
  91 | 301.37 | 01/05/2018 |  10.82 | avis       | Oak Park                                
  92 | 301.37 | 01/05/2018 |  31.61 | avis       | Streamwood, Il                          
  93 | 302.13 | 01/05/2018 |  22.76 | budget     | Elk Grove Village Il                    
  94 | 302.13 | 01/05/2018 |  25.02 | avis       | Renaissance Chicago North Shore         
  95 | 302.13 | 01/05/2018 |  15.03 | budget     | Lincolnwood                             
  96 | 302.13 | 01/05/2018 |  16.46 | budget     | Evanston                                
  97 | 302.13 | 01/05/2018 |  18.35 | budget     | Skokie                                  
  98 | 302.13 | 01/05/2018 |  21.15 | budget     | Niles Il Sears                          
  99 | 303.13 | 01/05/2018 |  28.09 | budget     | Hyatt Woodfield                         
 100 | 303.13 | 01/05/2018 |  30.75 | budget     | Schaumburg                              
 101 | 304.08 | 01/05/2018 |  22.76 | avis       | Elk Grove Village, Il                   
 102 | 304.08 | 01/05/2018 |  16.46 | avis       | Evanston (525 Kedzie Building)          
 103 | 304.08 | 01/05/2018 |  18.35 | avis       | Skokie - Dempster Street                
 104 | 304.08 | 01/05/2018 |  21.15 | avis       | Sears Auto Ctr - Golf Mill Shopping Ctr 
 105 | 305.08 | 01/05/2018 |  28.09 | avis       | Schaumburg Hyatt Regency Hotel          
 106 | 305.08 | 01/05/2018 |  30.75 | avis       | Schaumburg/hoffman Estates              
 107 | 314.42 | 01/05/2018 |  11.85 | enterprise | Nn Riverview                            
 108 | 314.42 | 01/05/2018 |  13.81 | enterprise | Napleton Chrysler                       
 109 | 314.42 | 01/05/2018 |  15.24 | enterprise | Nn Norwood Park C1                      
 110 | 315.32 | 01/05/2018 |  27.52 | enterprise | Aa Rolling Meadows Nt                   
 111 | 318.16 | 01/05/2018 |  27.34 | enterprise | Nn-arlington Heights                    
 112 | 319.16 | 01/05/2018 |  27.81 | enterprise | Aa East Schaumburg                      
 113 | 319.16 | 01/05/2018 |  28.43 | enterprise | Aa Roselle                              
 114 | 319.16 | 01/05/2018 |  28.87 | enterprise | Aa Schaumburg                           
 115 | 319.16 | 01/05/2018 |  30.76 | enterprise | Aa Hoffman Estates                      
 116 | 321.39 | 01/05/2018 |  21.60 | enterprise | Gg Gary                                 
 117 | 321.39 | 01/05/2018 |  21.81 | enterprise | Gg Schererville                         
 118 | 321.39 | 01/05/2018 |  36.52 | enterprise | Gg Valparaiso                           
 119 | 321.39 | 01/05/2018 |  24.13 | enterprise | Gg Hobart                               
 120 | 321.39 | 01/05/2018 |  14.69 | enterprise | Gg Hammond                              
 121 | 321.39 | 01/05/2018 |  26.33 | enterprise | Gg Merrillville                         
 122 | 321.39 | 01/05/2018 |  28.45 | enterprise | Gg East Merrillville                    
 123 | 321.39 | 01/05/2018 |  29.60 | enterprise | Gg Crown Point Indiana                  
 124 | 321.39 | 01/05/2018 |  30.13 | enterprise | Gg Cedar Lake                           
 125 | 321.39 | 01/05/2018 |  19.43 | enterprise | Gg Highland                             
 126 | 321.39 | 01/05/2018 |  19.55 | enterprise | Gg Griffith                             
 127 | 321.49 | 01/05/2018 |  23.02 | enterprise | Gg Steger                               
 128 | 321.49 | 01/05/2018 |  26.83 | enterprise | Gg New Lenox                            
 129 | 324.55 | 01/05/2018 |  22.13 | enterprise | Gg Frankfort And Mokena                 
 130 | 324.55 | 01/05/2018 |  36.65 | enterprise | Aa Aurora                               
 131 | 329.16 | 01/05/2018 |  10.15 | budget     | Logan Square Chicago Il                 
 132 | 329.16 | 01/05/2018 |  10.55 | budget     | Chicago/lakeview                        
 133 | 329.16 | 01/05/2018 |   5.11 | budget     | Chicago                                 
 134 | 329.16 | 01/05/2018 |   7.94 | budget     | Chicago Grossinger Autoplex             
 135 | 329.16 | 01/05/2018 |   4.05 | budget     | Chicago                                 
 136 | 331.27 | 01/05/2018 |  10.56 | avis       | Lakeview/chicago                        
 137 | 331.27 | 01/05/2018 |   5.71 | avis       | Chicago Union Train Station             
 138 | 331.27 | 01/05/2018 |  10.15 | avis       | Logan Square                            
 139 | 331.27 | 01/05/2018 |   4.05 | avis       | Chicago - 18th Street & Wabash Avenue   
 140 | 332.06 | 01/05/2018 |  37.57 | budget     | Libertyville                            
 141 | 332.83 | 01/05/2018 |  36.35 | enterprise | Aa Oswego                               
 142 | 337.38 | 01/05/2018 |  28.25 | enterprise | Gg Portage                              
 143 | 337.38 | 01/05/2018 |  28.80 | enterprise | Gg Arnell                               
 144 | 339.86 | 01/05/2018 |  22.80 | enterprise | Nn-glenview                             
 145 | 339.86 | 01/05/2018 |  12.70 | enterprise | Gg Brookfield                           
 146 | 339.86 | 01/05/2018 |   9.33 | enterprise | Gg Berwyn                               
 147 | 339.86 | 01/05/2018 |   9.47 | enterprise | Gg Blue Island                          
 148 | 339.88 | 01/05/2018 |  14.33 | enterprise | Gg Calumet City                         
 149 | 342.92 | 01/05/2018 |  21.92 | enterprise | Gg Miller Automall                      
 150 | 342.92 | 01/05/2018 |  11.09 | enterprise | Gg Bridgeview                           
 151 | 342.92 | 01/05/2018 |  11.20 | enterprise | Gg Crestwood                            
 152 | 342.92 | 01/05/2018 |  11.71 | enterprise | Gg Forest Park                          
 153 | 342.92 | 01/05/2018 |  13.76 | enterprise | Gg South Holland                        
 154 | 342.92 | 01/05/2018 |  16.75 | enterprise | Gg Orland Park East                     
 155 | 342.92 | 01/05/2018 |  17.26 | enterprise | Gg Homewood                             
 156 | 342.92 | 01/05/2018 |   8.70 | enterprise | Gg Oak Lawn                             
 157 | 342.92 | 01/05/2018 |  21.35 | enterprise | Gg Matteson                             
 158 | 342.94 | 01/05/2018 |  18.03 | enterprise | Gg Lansing                              
 159 | 345.67 | 01/05/2018 |  18.12 | enterprise | Gg Orland West                          
 160 | 350.50 | 01/05/2018 |  37.57 | avis       | Closing February 1, 2018                
 161 | 355.97 | 01/05/2018 |  17.63 | enterprise | Schiller Park                           
 162 | 361.81 | 01/05/2018 |   5.71 | budget     | Chicago Union Train Station             
 163 | 373.23 | 01/05/2018 |   4.19 | enterprise | Gg South Western                        
 164 | 373.23 | 01/05/2018 |  12.59 | enterprise | Nn Uptown                               
 165 | 373.23 | 01/05/2018 |   5.17 | enterprise | Gg Stony Island                         
 166 | 373.23 | 01/05/2018 |   5.57 | enterprise | Gg Gresham                              
 167 | 373.23 | 01/05/2018 |   5.82 | enterprise | Chicago Loop                            
 168 | 373.23 | 01/05/2018 |   5.84 | enterprise | Dd West Loop A/b(ex15ca)                
 169 | 373.23 | 01/05/2018 |   5.93 | enterprise | Union Station/pres Tower                
 170 | 373.23 | 01/05/2018 |   6.13 | enterprise | Gg Archer Heights                       
 171 | 373.23 | 01/05/2018 |   6.17 | enterprise | Gg Western And Roosevelt                
 172 | 373.23 | 01/05/2018 |   7.54 | enterprise | Dd Gold Coast Auto Body                 
 173 | 373.23 | 01/05/2018 |   7.87 | enterprise | Dd Old Town                             
 174 | 373.23 | 01/05/2018 |   8.91 | enterprise | Dd Bucktown                             
 175 | 373.23 | 01/05/2018 |   3.17 | enterprise | Gg South Loop                           
 176 | 373.23 | 01/05/2018 |   9.61 | enterprise | Nn Lincoln Park                         
 177 | 377.22 | 01/05/2018 |  23.87 | enterprise | Nn-mount Prospect                       
 178 | 377.22 | 01/05/2018 |  20.19 | enterprise | Nn-park Ridge                           
 179 | 377.22 | 01/05/2018 |  20.79 | enterprise | Nn Des Plaines 1505                     
 180 | 377.22 | 01/05/2018 |  21.00 | enterprise | Golf Mill Ford                          
 181 | 381.79 | 01/05/2018 |  37.77 | enterprise | Nn-north Chicago                        
 182 | 390.65 | 01/05/2018 |   0.95 | enterprise | Gg Hyde Park                            
 183 | 390.73 | 01/05/2018 |  26.05 | budget     | Merrillville                            
 184 | 399.70 | 01/05/2018 |  26.05 | avis       | Merrillville Car Care Center            
 185 | 402.87 | 01/05/2018 |   6.05 | budget     | North Loop Budget                       
 186 | 415.27 | 01/05/2018 |  39.53 | enterprise | Gg Michigan City                        
 187 | 419.44 | 01/05/2018 |   6.05 | avis       | North Loop-downtown                     
 188 | 444.98 | 01/05/2018 |   5.93 | enterprise | Millennium Park                         
 189 | 444.98 | 01/05/2018 |   6.09 | enterprise | Lake And Franklin                       
 190 | 444.98 | 01/05/2018 |   6.43 | enterprise | Chicago Magnificent Mile                
 191 | 444.98 | 01/05/2018 |   6.86 | enterprise | Gold Coast                              
 192 | 470.25 | 01/05/2018 |   6.07 | alamo      | Chicago N Lasalle                       
 193 | 582.19 | 01/05/2018 |   7.54 | budget     | Midway Airport                          
 194 | 644.31 | 01/05/2018 |  19.79 | enterprise | Ohare Intl Arpt                         
 195 | 660.23 | 01/05/2018 |  19.01 | budget     | Ohare Intl Airport                      
 196 | 711.77 | 01/05/2018 |  19.06 | alamo      | Chicago Ohare Airport                   
 197 | 736.25 | 01/05/2018 |  19.01 | avis       | Chicago Ohare Intl Airport              
 198 | 788.07 | 01/05/2018 |   7.60 | alamo      | Midway Intl Arpt                        
 199 | 796.04 | 01/05/2018 |   7.54 | avis       | Chicago Midway Intl Airport             
 200 | 887.09 | 01/05/2018 |   7.60 | enterprise | Midway Intl Arpt                        
"""

In [ ]:
res = session.query(Agency).filter(Agency.ag_id == 'E11509').first()
print(res)
res.prices

In [ ]:
import pprint
pprint.pprint(res)

In [ ]:
car1 = 'Economy Car'
car2 = 'Compact Car'
car3 = 'Intermediate Car'
car4 = 'Intermediate SUV'
desiredcar = car2
#desiredsort = 'time'
desiredsort = desiredcar
desiredbrand = 'enterprise'

# also, can optimize into some single query...buuuut it works as is
rvals = np.array([x for x in results.values() if x is not None and desiredcar in x and desiredsort in x])
orderpreferred = np.array([x[desiredsort] for x in rvals]).argsort()
orderprice = np.array([x[desiredcar] for x in rvals]).argsort()
order = [x for x in orderpreferred if x in orderprice[:60]]

In [ ]:
orderedresults = rvals[order]
print(desiredcar,car2,car3)
for k in orderedresults:
    #print(k['num'])
    if 'time' in k:
        print('{:s} | {:5.2f} | {:3d} | {:6.2f} | {:6.2f} | {:6.2f} | {:10s} | {}'
              .format(k['start']+'-'+k['end'],float(k['dist']),k['time']//60,k[desiredcar],k[car2],k[car3],str(k['brand']),k['name']))
    else:
        print('{:s} | {:5.2f} | {:3d} | {:6.2f} | {:6.2f} | {:6.2f} | {:10s} | {}'
              .format(k['start']+'-'+k['end'],float(k['dist']),0,k[desiredcar],k[car2],k[car3],str(k['brand']),k['name']))